# Cleaned Code

# Old Code

In [ ]:
!pip install osmnx
!pip install geopandas
!pip install -U spacy
!python -m spacy download en_core_web_lg
!pip install folium matplotlib mapclassify

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.9/102.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.6.1
    Uninstalling spacy-3.6.1:
      Successfully uninstalled spacy-3.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.6.0 requires spacy<3.7.0,>=3.6.0, but you have spacy 3.7.2 which is incompatible.
2023-12-05 04:47:19.635277: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-05 04:47:19.635344: E tensorflow/comp

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import spacy
import geopandas as gpd
import osmnx as ox

In [ ]:
nlp = spacy.load("en_core_web_lg")

In [ ]:
def filterGeolocations(text):
  doc = nlp(text)
  locations = [str(ent) for ent in doc.ents if ent.label_ == 'GPE']
  return locations

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/WWF x DSS/FinalLLMresultsAll.csv', index_col=0, encoding = "iso-8859-1", sep=",")

In [ ]:
df.head()

,PDF Name,Locations,Accomplishments
0,941417b4-e2a6-4337-b3e9-7bb73a02ce56.pdf,"Xayaboury, Cambodia, Mekong, United States, Vi...",\n\nThis report outlines a vision for a future...
1,b9b0aa20-292b-428f-90d3-f6c08cd5ae75.pdf,"Nepal, the United States Government",This plan outlines a strategy to secure 20 Gl...
2,a70e1021-627f-4a9d-86b1-5704a2a0891a.pdf,"Mozambique, United States, California",This article outlines a plan to improve infra...
3,acc1e526-4370-4134-97fe-e2951e314192.pdf,"China, Lower Mekong, Mekong basin, Pampas, Sek...",Argentina has the potential to become a globa...
5,9acd2c1b-3ac6-49ee-a000-2de92ed66c4f.pdf,"Bhutan, Himalayan, Jigme Dorji National Park, ...",This article provides an overview of a study ...


In [ ]:
def geocode(nameSeries, locSeries, accSeries):
  dfsList = []
  for name, loc, acc in zip(nameSeries, locSeries, accSeries):
    locList = filterGeolocations(loc)
    for spot in locList:
      try:
        area = ox.geocode_to_gdf(spot)
      except:
        print(f"disarding {spot}")
        continue
      area["Accomplishment"] = [acc]
      area["PDF Name"] = [name]
      dfsList.append(area)
  return pd.concat(dfsList, ignore_index=True)

locDf = geocode(df["PDF Name"], df["Locations"], df["Accomplishments"])

disarding Himalayas
disarding Sagarmatha
disarding Sagarmatha
disarding Himalaya
disarding Okomatapati
disarding Palearctic
disarding the United States
disarding the United States
disarding Himalaya
disarding West Sumatra Province
disarding Himalaya
disarding Galpagos Islands
disarding the United States
disarding Curripaco
disarding Chyulu Hills
disarding the United States
disarding Langtang
disarding Himalayas
disarding the United 
 States
disarding Matavn
disarding Curripaco
disarding United Arab Emerites
disarding Bosnia Herzegovinia
disarding USAID.GOV
disarding Nepal-HCV1
disarding the Orinoqua Region
disarding the United States
disarding the United States
disarding Heilong
disarding Rep Mongolia
disarding United Arab Emerites
disarding Bosnia Herzegovinia
disarding Himalaya
disarding Tibetan Plateau
disarding the United States
disarding Azunosa
disarding MTCO2e
disarding Margni
disarding the United States
disarding Himalaya
disarding Heilong
disarding east Africa
disarding Nakoda

In [ ]:
len(locDf)

1990

In [ ]:
locDf.to_csv('allJsons.csv')

## geojson in chunk of size 100 = 20 geojson files to later combine

In [ ]:
import geopandas as gpd
import json

# Split the GeoDataFrame into chunks of size 100 rows
chunk_size = 100
num_chunks = len(locDf) // chunk_size + 1

chunk_files = []  # To store the names of the generated chunk files

for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size
    chunk_gdf = locDf.iloc[start_idx:end_idx]

    # Write each chunk to a GeoJSON file
    file_name = f"/content/drive/MyDrive/WWF x DSS/GeoJson_Chunks/fullMapGeojson_chunk_{i}.geojson"
    chunk_gdf.to_file(file_name, driver='GeoJSON')

    chunk_files.append(file_name)

In [ ]:
# Initialize an empty GeoDataFrame
combined_gdf = gpd.GeoDataFrame()

for file_name in chunk_files:
    try:
        print(f"Processing file: {file_name}")  # Print the current file being processed
        chunk_gdf = gpd.read_file(file_name, dtype={'geometry': 'geometry'})
        combined_gdf = combined_gdf.append(chunk_gdf, ignore_index=True)
    except FileNotFoundError:
        pass  # Skip if file not found
    except ValueError as e:
        print(f"Error occurred while processing {file_name}: {e}")

Processing file: /content/drive/MyDrive/WWF x DSS/GeoJson_Chunks/fullMapGeojson_chunk_0.geojson


<ipython-input-14-331658983119>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_gdf = combined_gdf.append(chunk_gdf, ignore_index=True)


Processing file: /content/drive/MyDrive/WWF x DSS/GeoJson_Chunks/fullMapGeojson_chunk_1.geojson


<ipython-input-14-331658983119>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_gdf = combined_gdf.append(chunk_gdf, ignore_index=True)


Processing file: /content/drive/MyDrive/WWF x DSS/GeoJson_Chunks/fullMapGeojson_chunk_2.geojson


...

Processing file: /content/drive/MyDrive/WWF x DSS/GeoJson_Chunks/fullMapGeojson_chunk_18.geojson


<ipython-input-14-331658983119>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_gdf = combined_gdf.append(chunk_gdf, ignore_index=True)


Processing file: /content/drive/MyDrive/WWF x DSS/GeoJson_Chunks/fullMapGeojson_chunk_19.geojson


<ipython-input-14-331658983119>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_gdf = combined_gdf.append(chunk_gdf, ignore_index=True)


TypeError: ignored

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

# Assuming 'combined_gdf' contains columns with latitude and longitude data

# Create a geometry column from latitude and longitude coordinates
combined_gdf['geometry'] = combined_gdf.apply(lambda row: Point(row['lon'], row['lat']), axis=1)

# Convert the DataFrame to a GeoDataFrame with the newly created geometry column
combined_gdf = gpd.GeoDataFrame(combined_gdf, geometry='geometry')

# Specify the output file path
output_file = "/content/drive/MyDrive/WWF x DSS/combinedFullMapGeojson.geojson"

# Save GeoDataFrame to a GeoJSON file
combined_gdf.to_file(output_file, driver='GeoJSON')

print("GeoJSON file saved.")

GeoJSON file saved.


In [2]:
import json
import os

# Base directory and file pattern
base_dir = "/content/drive/MyDrive/DSS/WWF x DSS/GeoJson_Chunks"
file_pattern = "fullMapGeojson_chunk_{}.geojson"

# List to hold all features from all files
all_features = []

# Loop through each file and collect features
for i in range(0, 21):  # Assuming there are 20 files
    file_path = os.path.join(base_dir, file_pattern.format(i))
    print(f"Processing file: {file_path}")  # Print the current file being processed
    try:
        with open(file_path, 'r') as file:
            geojson = json.load(file)
            all_features.extend(geojson['features'])
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        continue  # Skip the file if not found

# Create a new GeoJSON structure
merged_geojson = {
    "type": "FeatureCollection",
    "features": all_features
}

# File to save the merged GeoJSON
output_file = "/content/drive/MyDrive/DSS/WWF x DSS/merged_geojson.geojson"
with open(output_file, 'w') as file:
    json.dump(merged_geojson, file)

print("Merged GeoJSON saved to:", output_file)

Processing file: /content/drive/MyDrive/DSS/WWF x DSS/GeoJson_Chunks/fullMapGeojson_chunk_0.geojson
Processing file: /content/drive/MyDrive/DSS/WWF x DSS/GeoJson_Chunks/fullMapGeojson_chunk_1.geojson
Processing file: /content/drive/MyDrive/DSS/WWF x DSS/GeoJson_Chunks/fullMapGeojson_chunk_2.geojson
Processing file: /content/drive/MyDrive/DSS/WWF x DSS/GeoJson_Chunks/fullMapGeojson_chunk_3.geojson
Processing file: /content/drive/MyDrive/DSS/WWF x DSS/GeoJson_Chunks/fullMapGeojson_chunk_4.geojson
Processing file: /content/drive/MyDrive/DSS/WWF x DSS/GeoJson_Chunks/fullMapGeojson_chunk_5.geojson
Processing file: /content/drive/MyDrive/DSS/WWF x DSS/GeoJson_Chunks/fullMapGeojson_chunk_6.geojson
Processing file: /content/drive/MyDrive/DSS/WWF x DSS/GeoJson_Chunks/fullMapGeojson_chunk_7.geojson
Processing file: /content/drive/MyDrive/DSS/WWF x DSS/GeoJson_Chunks/fullMapGeojson_chunk_8.geojson
Processing file: /content/drive/MyDrive/DSS/WWF x DSS/GeoJson_Chunks/fullMapGeojson_chunk_9.geojson


In [ ]:
combined_gdf

,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,class,type,place_rank,importance,addresstype,name,display_name,Accomplishment,PDF Name,geometry
0,19.658361,18.851729,101.896734,101.405818,238035846,relation,5831798,19.253614,101.618098,boundary,administrative,12,0.450010,county,Xayabury District,"Xayabury District, Sainyabuli Province, Laos",\n\nThis report outlines a vision for a future...,941417b4-e2a6-4337-b3e9-7bb73a02ce56.pdf,"POLYGON ((101.4058183 19.2254756, 101.4097135 ..."
1,14.690422,9.411096,107.627679,102.333828,253413211,relation,49898,12.543322,104.814491,boundary,administrative,4,0.695583,country,Cambodia,Cambodia,\n\nThis report outlines a vision for a future...,941417b4-e2a6-4337-b3e9-7bb73a02ce56.pdf,"POLYGON ((102.3338282 13.537473, 102.3348379 1..."
2,18.349965,17.958861,104.239215,103.786945,237837954,relation,3322157,18.154440,104.091661,natural,water,22,0.200010,water,Mekong River,"Mekong River, Ban Khonsavan, Pakkading Distric...",\n\nThis report outlines a vision for a future...,941417b4-e2a6-4337-b3e9-7bb73a02ce56.pdf,"POLYGON ((103.7869446 18.3414297, 103.7869982 ..."
3,71.588953,-14.760836,180.000000,-180.000000,77249893,relation,148838,39.783730,-100.445882,boundary,administrative,4,0.935691,country,United States,United States,\n\nThis report outlines a vision for a future...,941417b4-e2a6-4337-b3e9-7bb73a02ce56.pdf,"MULTIPOLYGON (((-180 51.7940888, -179.9926876 ..."
4,23.392692,7.692085,114.857258,102.143864,253747752,relation,49915,15.926666,107.965086,boundary,administrative,4,0.751195,country,Vietnam,Vietnam,\n\nThis report outlines a vision for a future...,941417b4-e2a6-4337-b3e9-7bb73a02ce56.pdf,"MULTIPOLYGON (((114.5524841 10.101774, 114.552..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1985,53.560815,8.665038,134.775456,73.499735,239664601,relation,270056,35.000074,104.999927,boundary,administrative,4,0.878827,country,China,China,This article discusses the effects of climate...,4491687e-fbc0-4475-b961-81f274c069b9.pdf,"MULTIPOLYGON (((73.4997347 39.3817402, 73.5000..."
1986,-4.345801,-18.039381,24.087636,11.460979,65579527,relation,195267,-11.877577,17.569124,boundary,administrative,4,0.721118,country,Angola,Angola,This report card provides an assessment of th...,6615e63c-9122-4bb0-ad4c-479bdecfee10.pdf,"MULTIPOLYGON (((11.4609793 -16.5543239, 11.462..."
1987,-17.778137,-26.907007,29.373887,19.998649,60971181,relation,1889339,-23.168178,24.592874,boundary,administrative,4,0.669811,country,Botswana,Botswana,This report card provides an assessment of th...,6615e63c-9122-4bb0-ad4c-479bdecfee10.pdf,"POLYGON ((19.9986486 -24.0970949, 19.999128 -2..."
1988,-16.963511,-28.969450,25.261729,11.528038,60698142,relation,195266,-23.233550,17.323111,boundary,administrative,4,0.687423,country,Namibia,Namibia,This report card provides an assessment of th...,6615e63c-9122-4bb0-ad4c-479bdecfee10.pdf,"POLYGON ((11.5280384 -17.589373, 11.532659 -17..."


### IF IT CRASHES, JUST COMBINE USING FILE PATH OF SMALLER CHUNKS

In [ ]:
import geopandas as gpd

# Initialize an empty GeoDataFrame
combined_gdf = gpd.GeoDataFrame()

# Iterate through all GeoJSON files and append their data to the GeoDataFrame
for i in range(20):  # Assuming 20 files from "fullMapGeojson_chunk_0.geojson" to "fullMapGeojson_chunk_19.geojson"
    file_name = f"/content/drive/MyDrive/WWF x DSS/GeoJson_Chunks/fullMapGeojson_chunk_{i}.geojson"

    try:
        print(f"Processing file: {file_name}")  # Print the current file being processed
        chunk_gdf = gpd.read_file(file_name)
        combined_gdf = combined_gdf.append(chunk_gdf, ignore_index=True)
    except FileNotFoundError:
        pass  # Skip if file not found

# Write the combined GeoDataFrame to a single GeoJSON file
output_file = "/content/drive/MyDrive/WWF x DSS/combinedFullMapGeojson.geojson"
combined_gdf.to_file(output_file, driver='GeoJSON')

print("Merging completed. GeoJSON file saved.")